In [94]:
import pandas as pd 
import numpy as np
import datetime

main_data = pd.read_csv('ring_occupancy.csv',header=None)
main_data.columns = ['Initial_time', 'Final_time', 'Cage','ID','RFID']

# calculate duration: 

main_data['Cage']=main_data['Cage']+1

main_data.sort_values(by=['Initial_time'])

main_data['Initial_time']=main_data['Initial_time']/(1000*60)
main_data['Final_time']=main_data['Final_time']/(1000*60)

difference=main_data['Final_time']- main_data['Initial_time'] # we calculate the time difference between start and end of events 
main_data['Initial_time']=main_data['Initial_time']- main_data['Initial_time'][0] # scale the start time of events in relation to the very first event 
main_data['Final_time']=main_data['Initial_time']+difference
main_data['ID']=main_data['ID'].astype(np.int64)
main_data['Cage']=main_data['Cage'].astype(np.int64)

new_ID=[str(main_data['ID'][i])[-5:] for i in np.arange(len(main_data['ID']))]
main_data['ID']=new_ID
main_data['event_duration']=main_data['Final_time']-main_data['Initial_time']

initial_time_stamps=pd.read_csv('initial_event_time_stamps.txt',header=None)
end_time_stamps=pd.read_csv('end_event_time_stamps.txt',header=None)
mod_data = pd.concat([main_data,initial_time_stamps,end_time_stamps], axis=1, sort=False)

# use a nice list comprehension
mod_data.columns = ['Initial_time', 'Final_time', 'Cage','ID','RFID','event_duration','Event_start_ts','Event_end_ts']
mod_data


mod_data['Event_start_ts']=pd.to_datetime(mod_data['Event_start_ts'])
mod_data['Event_end_ts']=pd.to_datetime(mod_data['Event_end_ts'])

#type(mod_data['Event_start_ts'])
#formatted_times_start=[''.join(mod_data['Event_start_ts'][i].rsplit('.', 1)[0]) for i in np.arange(len(mod_data)) ]
#formatted_times_end=[''.join(mod_data['Event_end_ts'][i].rsplit('.', 1)[0]) for i in np.arange(len(mod_data)) ]

#date_time_objects_start=[datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in formatted_times_start]
#date_time_objects_end=[datetime.datetime.strptime(time, '%Y-%m-%d %H:%M:%S') for time in formatted_times_end]


#mod_data['Event_start_ts']=date_time_objects_start
#mod_data['Event_end_ts']=date_time_objects_end

# the first thing is to format things into the right time


# the problem here is that some reads contain milliseconds, and others don't. 
# the best thing to do is to split them and then rejoin them
#print(mod_data)
# no more NAs in the dataframe 



# once, you have the beginning and ending time stamps

# first, you need to merge all of this into a pandas dataframe. 
# using that dataframe, you should then begin adding all the bells and whistles so everything 
# is modified. 

# this would be analogous to your data massaging function in which you would add specific things like time 

In [95]:
mod_data['hrs']=mod_data["Event_start_ts"].apply(lambda x: x.hour)
# find the first element where it's greater than 0. 

In [96]:
#mod_data['Event_start_ts'].day

number_of_divisions=(mod_data['Event_start_ts'].iloc[-1].day-mod_data['Event_start_ts'].iloc[1].day)*(24)/(6)

In [97]:
# not just this. Find out the first element

time_range=pd.date_range('2018-06-20 12:00:00', periods=number_of_divisions+1, freq='360T')
time_range


DatetimeIndex(['2018-06-20 12:00:00', '2018-06-20 18:00:00',
               '2018-06-21 00:00:00', '2018-06-21 06:00:00',
               '2018-06-21 12:00:00', '2018-06-21 18:00:00',
               '2018-06-22 00:00:00', '2018-06-22 06:00:00',
               '2018-06-22 12:00:00', '2018-06-22 18:00:00',
               '2018-06-23 00:00:00', '2018-06-23 06:00:00',
               '2018-06-23 12:00:00', '2018-06-23 18:00:00',
               '2018-06-24 00:00:00', '2018-06-24 06:00:00',
               '2018-06-24 12:00:00', '2018-06-24 18:00:00',
               '2018-06-25 00:00:00', '2018-06-25 06:00:00',
               '2018-06-25 12:00:00', '2018-06-25 18:00:00',
               '2018-06-26 00:00:00', '2018-06-26 06:00:00',
               '2018-06-26 12:00:00', '2018-06-26 18:00:00',
               '2018-06-27 00:00:00', '2018-06-27 06:00:00',
               '2018-06-27 12:00:00', '2018-06-27 18:00:00',
               '2018-06-28 00:00:00', '2018-06-28 06:00:00',
               '2018-06-

In [98]:
# using a loop here, assign bins 

import numpy as np

for i in np.arange(len(time_range)-1):
    if i<len(time_range)-1:
        mod_data.loc[(mod_data['Event_start_ts'] >= time_range[i]) 
                 & (mod_data['Event_start_ts'] < time_range[i+1]),'Bin']=i
    else: 
        mod_data.loc[(mod_data['Event_start_ts'] >= time_range[i]),'Bin']=i   

In [99]:
mod_data.iloc[130000]

Initial_time                         12251.6
Final_time                           12252.5
Cage                                       9
ID                                     90239
RFID                                       0
event_duration                      0.903333
Event_start_ts    2018-06-28 23:35:26.809000
Event_end_ts      2018-06-28 23:36:21.009000
hrs                                       23
Bin                                       33
Name: 130000, dtype: object

In [100]:
CD1_metadata= pd.read_csv('CD1_metadata.csv')

In [101]:
CD1_metadata.columns=['ID','RFID_tag','Gender','Litter']

In [102]:
# to merge this data, you need to have the data sorted. 

#mod_data=mod_data.sort_values('ID')

d=pd.value_counts(mod_data['ID']).to_frame().reset_index()

d=d.sort_values('index')
d

d.columns=['ID','Count']
d

,ID,Count
13,80821,3530
14,80853,1127
2,81121,13803
0,81403,19346
11,81561,5063
5,82326,9509
4,86789,11444
10,89482,7012
7,89649,8805
6,90239,9084


In [103]:
CD1_metadata

print(CD1_metadata.dtypes)

ID           int64
RFID_tag    object
Gender      object
Litter      object
dtype: object


In [104]:
d['ID']=d['ID'].astype('int64')

In [105]:
CD1_metadata=CD1_metadata.merge(d)

In [106]:
CD1_metadata

,ID,RFID_tag,Gender,Litter,Count
0,80821,2A006D2AD5B8,M,F11L1,3530
1,80853,2A006D2AF598,M,F11L1,1127
2,81121,2A006D2C016A,F,F02L1,13803
3,81403,2A006D2D1B71,M,F15L1,19346
4,81561,2A006D2DB9D3,M,F15L1,5063
5,82326,2A006D30B6C1,M,F15L1,9509
6,86789,2A006D422520,F,F11L1,11444
7,89482,2A006D4CAAA1,F,F17L1,7012
8,89649,2A006D4D515B,M,F15L1,8805
9,90239,2A006D4F9F97,F,F11L1,9084


In [139]:
mod_data_final=pd.merge(mod_data,CD1_metadata, copy=False)
#mod_data_final=mod_data_final.drop_duplicates() 

,Initial_time,Final_time,Cage,ID,RFID,event_duration,Event_start_ts,Event_end_ts,hrs,Bin,RFID_tag,Gender,Litter,Count
0,0.000000,0.411583,10,89482,0.0,0.411583,2018-06-20 11:23:51.531,2018-06-20 11:24:16.226,11,NaN,2A006D4CAAA1,F,F17L1,7012
1,0.411583,1.649200,9,89482,0.0,1.237617,2018-06-20 11:24:16.226,2018-06-20 11:25:30.483,11,NaN,2A006D4CAAA1,F,F17L1,7012
2,1.649200,1.990767,8,89482,0.0,0.341567,2018-06-20 11:25:30.483,2018-06-20 11:25:50.977,11,NaN,2A006D4CAAA1,F,F17L1,7012
3,1.990767,2.137600,9,89482,0.0,0.146833,2018-06-20 11:25:50.977,2018-06-20 11:25:59.787,11,NaN,2A006D4CAAA1,F,F17L1,7012
4,2.644583,2.856367,9,89482,0.0,0.211783,2018-06-20 11:26:30.206,2018-06-20 11:26:42.913,11,NaN,2A006D4CAAA1,F,F17L1,7012
5,2.856367,2.890867,8,89482,0.0,0.034500,2018-06-20 11:26:42.913,2018-06-20 11:26:44.983,11,NaN,2A006D4CAAA1,F,F17L1,7012
6,2.890867,3.901633,9,89482,0.0,1.010767,2018-06-20 11:26:44.983,2018-06-20 11:27:45.629,11,NaN,2A006D4CAAA1,F,F17L1,7012
7,3.901633,4.033117,8,89482,0.0,0.131483,2018-06-20 11:27:45.629,2018-06-20 11:27:53.518,11,NaN,2A006D4CAAA1,F,F17L1,7012
8,4.033117,4.227600,7,89482,0.0,0.194483,2018-06-20 11:27:53.518,2018-06-20 11:28:05.187,11,NaN,2A006D4CAAA1,F,F17L1,7012
9,4.227600,4.749300,8,89482,0.0,0.521700,2018-06-20 11:28:05.187,2018-06-20 11:28:36.489,11,NaN,2A006D4CAAA1,F,F17L1,7012


In [ ]:
# now, the next step is to actually bin the data 